# Exercício 07_01

## Árvore

Criar o código necessários para que os métodos `.cria_regras()` e `.predicao()` cumpram seus objetivos. O primeiro deve percorrer a árvore já criada pelo `.cria_galhos()` e armazeanda nos atributos `filhos`, armazenando em um dicionário as regras para cada quebra. Já o segundo deve para uma nova instância, percorrer as regras criadas pelo método anterior e fazer a predição baseado nisso.

Você pode passar nos parâmetros para os métodos que julgar necessário.

Lembrando que partimos das premissas que:

- Nosso problema terá como variáveis explicativas apenas dados categóricos;
- Nosso problema terá como variável resposta apenas dados categóricos;

__Passo 0:__ Importando as bibliotecas necessárias para o processamento.

In [1]:
import pandas as pd
import numpy as np

__Passo 1:__ Criar a classe Árvore, a qual realizará o cálculo da entropia original, o ganho de informação de cada atributo, realiza a quebra dos galhos de acordo com o atributo com maior ganho de informação, repete os passos anteriores até chegar à menor profundidade solicitada.

In [47]:
class Arvore:
    
    def __init__(self, 
                 df, var_resposta, 
                 atributo_origem = '-',
                 atributo_origem_valor = '-', 
                 profundidade = 0):
        
        self.df = df
        self.var_resposta = var_resposta
        self.filhos = []
        self.atributo_melhor = '-'
        self.atributo_origem = atributo_origem
        self.atributo_origem_valor = atributo_origem_valor
        self.gi = 0
        self.profundidade = profundidade
    
    def __repr__(self):
        return f'''Nó {self.profundidade} da Árvore de Decisão.\nCaracterísticas do Nó:
            Dataframe de {self.df.shape[0]} linhas.
            A origem desse nó é o atributo {self.atributo_origem} com valor {self.atributo_origem_valor}
            Melhor atributo desse nó é {self.atributo_melhor} com GI de {self.gi}'''
    
    @staticmethod
    def entropia_binaria(p):
    
        if p == 0 or p == 1:
            return 0

        n = 1 - p
        return -(p*np.log2(p) + n*np.log2(n))
    
    @staticmethod
    def ganho_informacao(df, coluna, var_resposta):
    
        seq_poss = df[coluna].unique()

        ent_inicial = Arvore.entropia_binaria(df[var_resposta].value_counts(1).iloc[0])

        conj = [df[df[coluna] == i] for i in seq_poss]
        ent = [Arvore.entropia_binaria(i[var_resposta].value_counts(1).iloc[0]) for i in conj]
        n_elem = [i.shape[0] for i in conj]

        total = sum(n_elem)

        ent_atr = 0
        for i in range(len(conj)):
            ent_atr += ent[i]*n_elem[i]/total

        return ent_inicial - ent_atr
        
    def melhor_atributo(self):
        '''
        Esse método acha o melhor atributo em relação ao ganho de informação.
        '''

        colunas_a_testar = self.df.columns.to_list()
        colunas_a_testar.remove(self.var_resposta)

        maior_v = 0
        maior_k = '-'

        for col in colunas_a_testar:

            v = Arvore.ganho_informacao(self.df, col, self.var_resposta)
            if v > maior_v:
                maior_v = v
                maior_k = col
        
        return maior_k, maior_v
        
    def realiza_quebra(self):
        '''
        Quebra o conjunto de dados inicial em N subconjuntos de acordo com 
        o melhor atributo.
        '''
        
        self.atributo_melhor, self.gi = self.melhor_atributo()
        
        if self.atributo_melhor == '-':
            return 
        
        possibilidades = self.df[self.atributo_melhor].unique()
        
        for p in possibilidades:
            mascara = self.df[self.atributo_melhor] == p
            sub_conj = self.df.loc[mascara, :]
            self.filhos.append(Arvore(sub_conj, 
                                      self.var_resposta, 
                                      atributo_origem = self.atributo_melhor,
                                      atributo_origem_valor = p,
                                      profundidade = self.profundidade + 1))            
        
        
    def cria_galhos(self):
        
        self.realiza_quebra()
        
        for i in self.filhos:
            
            p0 = i.df[i.var_resposta].value_counts(1).iloc[0]
            ent = Arvore.entropia_binaria(p0)
            """
            Inseri novos códigos a partir daqui
            """
            #mascara = self.df[self.atributo_melhor] == p
            #sub_conj = self.df.loc[mascara, :]
            #self.filhos.append(Arvore(sub_conj, 
            #                          self.var_resposta, 
            #                          atributo_origem = self.atributo_melhor,
            #                          atributo_origem_valor = p,
            #                          profundidade = self.profundidade + 1))

            """
            Até aqui
            """

            if ent > 0:
                i.cria_galhos()
    
    # TODO
    def cria_regras(self):
        '''
        Esse método deve percorrer recursivamente as árvores retornando o
        conjunto de regras que definem a qual nó a nova instância pertencerá.
        '''
        for i in self.filhos:
            print(i)
            
        # Condição de parada:
        if self.gi == 0:
            return
                
        return regras
    
    # TODO
    def predicao(self, nova_instancia):
        ''' 
        Dado uma nova instância, retorna a sua predição.
        '''
        
        
        return pred

__Passo 2:__ Importando o dataset e criando uma amostra aleatória para servir como base de treinamento.

In [48]:
df = pd.read_csv('titanic.csv')

df_amostra = df.sample(15, random_state = 1)

__Passo 3:__ Realizando a primeira execução da classe árvore na base de treinamento.

In [49]:
arv = Arvore(df_amostra[['Survived', 'Parch', 'Sex', 'SibSp', 'Pclass']], 'Survived')

<center> <h3> IMPORTANTE! <\h3> <\center>

Neste momento, apenas a raiz da árvore de decisão foi criado. Como pode ser verificado buscando os filhos da árvore, conforme abaixo:

In [50]:
arv.filhos

[]

__Passo 4:__ Agora devemos executar a função de cálculo do melhor atributo para identificar qual a coluna que apresenta maior ganho de informação em nosso dataframe. 

In [51]:
arv.melhor_atributo()

('Sex', 0.430776632270099)

__Passo 5:__ Com base no atributo de maior ganho de informação, descoberto por meio do __Passo 4__, realizamos a quebra da raiz em dois galhos, conforme abaixo:

In [52]:
arv.cria_galhos()

<center> <h3> Importante! <\h3><\center>

Neste momento, existem dois galhos criados que estão como filhos da árvore, conforme pode ser verificado abaixo:

In [60]:
arv

2

In [33]:
for i in arv.filhos:
            
    p0 = i.df[i.var_resposta].value_counts(1).iloc[0]

In [34]:
p0

0.8571428571428571

In [ ]:
arv.predicao(df_amostra.iloc[0, :])